In [2]:
import sys,os
sys.path.append('../')

In [3]:
from unit.dataset_multiclass import get_dataloaders
from unit.model import ContrastiveLearningModel
from unit.config import config
from datasets import load_from_disk
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch
import pandas as pd
from tqdm.auto import tqdm
import torch.nn.functional as F
from adapters import AutoAdapterModel

In [4]:
pretrain_model = AutoAdapterModel.from_pretrained(config.MODEL_NAME)
model = ContrastiveLearningModel(pretrain_model, 
                                learning_rate=config.LEARNING_RATE, 
                                weight_decay=config.WEIGHT_DECAY)

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/user_data/envs/adapter/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:208: Attribute 'adapter_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['adapter_model'])`.


In [5]:
from unit.datasets import TextDataset, MultiLabelTextDataset

from transformers import AutoTokenizer

base_model_name = 'FacebookAI/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(base_model_name) # 加载预训练的模型和 tokenizer    

In [6]:
# 預測產生結果、多類別 Multi-Class
def MultiClassPredit(model, dataloader, tokenizer, device):
    model.to(device)
    model.eval()
    y_probas = []
    preds = []
    labels = []
    texts = []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids, attention_mask, label = [t.to(device) for t in batch]
            # print(input_ids, attention_mask, label)
            output = model(input_ids=input_ids, attention_mask=attention_mask)
            pred = torch.argmax(output, dim=1)
            y_proba = F.softmax(output, dim=-1)

            preds.extend(pred.round().cpu().numpy())            
            y_probas.extend(y_proba.cpu().numpy())
            labels.extend(label.cpu().numpy())

            #print(input_ids)
            text = tokenizer.batch_decode(input_ids.cpu(),skip_special_tokens=True)
            #print(text)
            texts.extend(text)

    return np.array(texts), np.array(y_probas), np.array(preds), np.array(labels)

In [7]:
def save(base_model_name,test_texts,y_true,y_pred,test_probs,file_name='prediction.csv'):
    output_dir = os.path.join(config.PROJECT_DIR,
                                'models',
                                config.PROJECT_NAME,
                                'init',
                                config.VERSION,
                                config.CONTRASTIVE_STRATEGY, # random or mix                            
                                base_model_name,
                                'results')

    # 保存结果到模型路径中的 results 目录
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    
    prediction_df = pd.DataFrame([],columns=['text','y_pred','y_true','y_proba'])
    
    prediction_df['text'] = test_texts.tolist()    
    prediction_df['y_pred'] = y_pred
    prediction_df['y_true'] = y_true    
    prediction_df['y_proba'] = test_probs.tolist()

    
    prediction_df.to_csv(f'{output_dir}/{file_name}',index=None) 

In [8]:
# 进行推理和评估
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

label_encoder = LabelEncoder()
dataset_test = load_from_disk(f'{config.PROJECT_DIR}/data/NLP4IF2019/Encoder-based-hierarchical/test/dataset') 
test_texts = dataset_test['test']['text']
test_labels = dataset_test['test']['classes']

In [14]:
df = pd.DataFrame(test_labels,columns=['classes'])

In [23]:
classes_count = df['classes'].value_counts().sort_index(ascending=True).tolist()
total_num = len(df)

print(total_num)
print(classes_count)

5079
[3572, 330, 220, 545, 60, 61, 291]


In [30]:
init_weight = [0.25,0.75,0.75,0.75,0.75,0.75,0.75]
frequency_rate = [x/total_num for x in classes_count]
frequency_rate[0] = 1-frequency_rate[0]
print(frequency_rate)

frequency_gap = [a*b for a,b in zip(frequency_rate,init_weight)]
weight = [b-a for a,b in zip(frequency_gap,init_weight)]

print(weight)

[0.2967119511714904, 0.06497341996455995, 0.04331561330970664, 0.1073045875172278, 0.011813349084465446, 0.012010238235873204, 0.05729474305965741]
[0.1758220122071274, 0.7012699350265801, 0.71751329001772, 0.6695215593620791, 0.7411399881866509, 0.7409923213230951, 0.707028942705257]


In [7]:




test_labels_encoder = label_encoder.fit_transform(test_labels)  

# 创建数据集实例        
test_dataset = TextDataset(test_texts, test_labels_encoder, tokenizer)

# 创建 DataLoader
num_workers = 19  # 设置 num_workers 参数以提高性能
test_loader = DataLoader(test_dataset, batch_size=4, num_workers=num_workers) 

# np.array(texts), np.array(y_probas), np.array(preds), np.array(labels)
test_texts, test_probs, test_preds, test_labels  = MultiClassPredit(model, test_loader, tokenizer, device) 

y_pred = test_preds.astype(int).tolist()
y_true = test_labels.tolist()

save(base_model_name,test_texts,y_true,y_pred,test_probs,file_name='prediction_total.csv')

  0%|          | 0/1270 [00:01<?, ?it/s]

In [ ]:
test_labels

NameError: name 'test_labels' is not defined

Eval

In [ ]:
project = 'PersuTech'
model_name_or_path = 'FacebookAI/roberta-large'
task_name = "init"

In [ ]:
!{sys.executable} ../script/evaluate_multiclass.py --project {project} \
--task_name {task_name} \
--version 'v1' \
--strategy 'random' \
--base_model_name {model_name_or_path} \
--dataset_path_or_name 'prediction_total.csv' 

/user_data/envs/cyberbullying/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/user_data/envs/cyberbullying/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/user_data/envs/cyberbullying/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Class 0 Ac